SVM 분석

In [159]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report 

df_new = df.drop(['steel_kind', 'hsb', 'rolling_method', 'furnace'], axis=1) 
df_new

,scale,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,rolling_temp,descaling_count,fur_preheat_time
0,0,32,3700,15100,1144,116,1133,59,934,8,84
1,0,32,3700,15100,1144,122,1135,53,937,8,63
2,0,33,3600,19200,1129,116,1121,55,889,8,87
3,0,33,3600,19200,1152,125,1127,68,885,8,73
4,0,38,3100,13300,1140,134,1128,48,873,8,64
...,...,...,...,...,...,...,...,...,...,...,...
989,0,19,3400,41500,1142,55,1151,86,948,10,62
990,0,19,3400,41500,1142,55,1151,86,948,10,68
991,0,17,3400,43700,1169,65,1163,77,948,10,85
992,0,17,3400,43700,1169,65,1163,77,948,10,62


In [160]:
# 데이터 분리:설명변수 스케일 변환
df_raw_x = df_new.drop("scale", axis = 1, inplace = False)
df_raw_y = df_new["scale"] 

v_feature_names = df_raw_x.columns
scaler = StandardScaler() 
df_x_scaled = scaler.fit_transform(df_raw_x)
df_x_scaled = pd.DataFrame(df_x_scaled, columns = v_feature_names)
dropped_columns = df[['steel_kind', 'hsb', 'rolling_method', 'furnace']] 
df_x_combined = df_x_scaled.join(dropped_columns)
df_x_combined
df_raw_y = df_raw_y.to_frame()
df_x_converted = df_raw_y.join(df_x_combined)

df_raw_dummy_final = pd.get_dummies(df_x_converted)
df_raw_dummy_final.head()

df_raw_x_final = df_raw_dummy_final.drop("scale", axis = 1, inplace = False)
df_raw_y_final = df_raw_dummy_final["scale"] 

display(df_raw_dummy.head())

,scale,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,rolling_temp,descaling_count,...,hsb_미적용,hsb_적용,rolling_method_CR(제어압연),rolling_method_TMCP(온도제어),furnace_1호기1열,furnace_1호기2열,furnace_2호기1열,furnace_2호기2열,furnace_3호기1열,furnace_3호기2열
0,0,0.297380,1.758916,-1.568542,-0.629006,1.138138,-1.042290,-0.617512,-0.098376,-0.353037,...,0,1,0,1,1,0,0,0,0,0
1,0,0.297380,1.758916,-1.568542,-0.629006,1.365628,-0.926564,-0.908186,-0.051371,-0.353037,...,0,1,0,1,0,1,0,0,0,0
2,0,0.352984,1.555807,-1.273013,-1.336122,1.138138,-1.736646,-0.811295,-0.803452,-0.353037,...,0,1,0,1,0,0,1,0,0,0
3,0,0.352984,1.555807,-1.273013,-0.251878,1.479374,-1.389468,-0.181501,-0.866125,-0.353037,...,0,1,0,1,0,0,0,1,0,0
4,0,0.631008,0.540262,-1.698287,-0.817571,1.820609,-1.331605,-1.150415,-1.054145,-0.353037,...,0,1,0,1,0,0,0,0,1,0


In [142]:
# 데이터 분할 train_test_split(X: 설명변수, Y: 목표변수, test_size = test 데이터 비율)
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x_final, df_raw_y_final, test_size = 0.3, random_state = 1234) 

print("분할 전 설명변수 데이터 :", df_x_scaled.shape)
print("분할 후 설명변수 데이터 :Train", df_train_x.shape, "  Test",df_test_x.shape)

분할 전 설명변수 데이터 : (994, 46)
분할 후 설명변수 데이터 :Train (695, 58)   Test (299, 58)


In [150]:
df_train_y.value_counts()

0    473
1    222
Name: scale, dtype: int64

In [161]:
from imblearn.over_sampling import SMOTE
# Over-sampling 설정
sm = SMOTE(sampling_strategy='auto', random_state=1234)


# train데이터를 이용한 Over-sampling
x_resampled, y_resampled = sm.fit_resample(df_train_x, df_train_y)

y_resampled.value_counts()

1    473
0    473
Name: scale, dtype: int64

In [170]:
para_c = [10**c for c in range(-2, 3)]

para_gamma = [10**gamma for gamma in range(-2, 3)]

estimator = SVC(random_state=1234)

param_grid = {"C":para_c, "gamma":para_gamma}

grid_rf = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_rf.fit(x_resampled, y_resampled)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

best estimator model: 
SVC(C=100, gamma=0.1, random_state=1234)

best parameter: 
{'C': 100, 'gamma': 0.1}

best score: 
0.924


train 모델 기준 최적 파라미터로 C=100, gamma=0.1 추천, 정확도는 92.4%

In [172]:
svc_final = SVC(gamma=0.1, C=100, random_state=1234)
svc_final.fit(df_train_x, df_train_y)
y_pred = svc_final.predict(df_test_x)

print("Train Accuracy: {:.3f}".format(svc_final.score(df_train_x, df_train_y)))
print("Test Accuracy: {:.3f}\n".format(svc_final.score(df_test_x, df_test_y)))
print("Test confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
print(classification_report(df_test_y, y_pred, digits=3))

Train Accuracy: 1.000
Test Accuracy: 0.860

Test confusion matrix: 
[[189  22]
 [ 20  68]]
              precision    recall  f1-score   support

           0      0.904     0.896     0.900       211
           1      0.756     0.773     0.764        88

    accuracy                          0.860       299
   macro avg      0.830     0.834     0.832       299
weighted avg      0.861     0.860     0.860       299



test 데이터 기준 정분류율은 86%, "1"기준 f1 score는 76.4% 